## Importing Libraries

In [4]:
from ollama import chat
from ollama import ChatResponse
from ollama import embed
import json

In [5]:
EMBEDDING_MODEL = "mxbai-embed-large"
GENERATION_MODEL = "qwen3:1.7b"

In [6]:
def cosine_similarity(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

In [7]:
def retrieve(query, top_n=3):
    query_embedding = embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]

    with open("../Vector-store/vector_db.json", "r", encoding="utf-8") as f:
        raw = json.load(f)
        VECTOR_DB = [(item["chunk"], item["embedding"]) for item in raw]

    # temporary list to store (chunk, similarity) pairs
    similarities = []

    for chunk, embedding in VECTOR_DB:
        # embedding is already a list of floats (from your JSON)
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))

    # Sort after finishing loop
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities[:top_n]

In [13]:
input_query = "What relationship exists between CARAFE and object detection?"
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')


instruction_prompt = f'''instruction_prompt =
Answer using the context only.
Then provide a step-by-step justification under the section "JUSTIFICATION",
where each step must explicitly reference which context snippet supports it.

You can reveal chain-of-thought. Don't make up any new information: {'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}'''

Retrieved knowledge:
 - (similarity: 0.83) To demonstrate the universal effectiveness of CARAFE , we conduct comprehensive evaluations across a wide range of dense prediction tasks , i.e. , object detection , instance segmentation , semantic segmentation , image inpainting , with mainstream architectures .
 - (similarity: 0.79) CARAFE can boost the performance of Faster RCNN [ 3 0 ] [ 4 3 , 4 4 ] val in semantic segmentation , and improves Global&Local [ 1 1 ] by 1. 1 dB of PSNR on Places [ 4 2 ] val in image inpainting .
 - (similarity: 0.79) With negligible additional parameters , CARAFE benefits state - of - the - art methods in both highlevel and low - level tasks , such as object detection , instance segmentation , semantic segmentation and image inpainting .


In [14]:
stream = chat(
    model=GENERATION_MODEL, 
    messages=[
        {
            'role': 'system', 
            'content': instruction_prompt
        },
        {
            'role': 'user',
            'content': input_query
        },
        ],
        stream=True,
        )
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
The relationship between CARAFE and object detection is that CARAFE is evaluated across dense prediction tasks, including object detection, and is shown to improve performance in tasks like semantic segmentation (which is a subset of object detection). 

**JUSTIFICATION:**  
1. The first context snippet states: "To demonstrate the universal effectiveness of CARAFE, we conduct comprehensive evaluations across a wide range of dense prediction tasks, i.e., object detection, instance segmentation, semantic segmentation, image inpainting."  
   - This explicitly links CARAFE to object detection as one of the evaluated tasks.  

2. The second context snippet mentions: "CARAFE can boost the performance of Faster RCNN [30][43,44] val in semantic segmentation, and improves Global&Local [11] by 1.1 dB of PSNR on Places [42] val in image inpainting."  
   - Semantic segmentation is a subtask of object detection, and CARAFE's improvement in this task demonstrates its effectivenes